## The Most Common Use Pattern for Pandas `apply()`

> You might be surprised how many people can't write `apply()`.

> Avoid `apply()` cuz it's slow and memory inefficient. 

These are statements from data scientists I respect. They are both correct. On
one hand, you want to know `apply()` because it's convenient and because it can 
show up during interviews. On the other hand, `apply()` is often not needed, 
and there are alternatives that are faster and more memory efficient.

My goal here is to show you the most common use pattern of `apply()`, while 
ignoring the many other things it can do. (Yes, it can do many things, but 
always remember the 80-20 rule even when coding.)

*I primarily use `apply()` to apply a reducing function (often for aggregation purpose)
either column-wise (axis=0, default) or row-wise (axis=1) on a DataFrame or 
a group of DataFrames when the dataset is small or medium-sized.*

By reducing function, I mean a function that collapses the dimension of the input data. 
Let's see some examples.

In [1]:
import pandas as pd
import numpy as np
from defillama2 import DefiLlama

### Data Prep

In [2]:
obj = DefiLlama() # create a DefiLlama instance
df = obj.get_protocols_fundamentals() # get fundamentals for all protocols
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3079 entries, 0 to 3078
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         3079 non-null   object 
 1   symbol       3079 non-null   object 
 2   chain        3079 non-null   object 
 3   category     3079 non-null   object 
 4   chains       3079 non-null   object 
 5   tvl          3079 non-null   float64
 6   change_1d    2514 non-null   float64
 7   change_7d    2456 non-null   float64
 8   mcap         1423 non-null   float64
 9   forked_from  2236 non-null   object 
dtypes: float64(4), object(6)
memory usage: 240.7+ KB


In [3]:
cond = df.category == 'Liquid Staking' # focus on LSD protocols
cols = ['name', 'tvl', 'change_1d', 'change_7d', 'mcap'] # focus on these cols
subdf = df.loc[cond, cols].reset_index(drop=True) # drop the original int index
subdf.set_index('name', inplace=True) # use name as index
subdf.head()

,tvl,change_1d,change_7d,mcap
name,,,,
Lido,1.473266e+10,0.464218,-1.701088,1.686376e+09
Coinbase Wrapped Staked ETH,2.260730e+09,0.254382,-1.876672,NaN
Rocket Pool,1.856144e+09,0.220902,-2.665735,5.787441e+08
Frax Ether,4.497888e+08,0.368398,-1.441630,NaN
StakeWise,1.806842e+08,0.402752,-2.010111,2.619703e+07


### A Light Intro

In [4]:
# check how many values are missing in each column
subdf.isna().apply('sum', axis=0) # axis=0 means apply the function column wise, i.e., for each col

tvl           0
change_1d    12
change_7d    13
mcap         80
dtype: int64

In [5]:
# but please don't write that in practice, do this instead
subdf.isna().sum(axis=0)

tvl           0
change_1d    12
change_7d    13
mcap         80
dtype: int64

Remark: `sum()` is an aggregation function, and aggregation functions reduce data dimensions. Run `subdf.isna()` and check its output's shape and compare with the shape of the above output. 

In [6]:
# calculate the range of each col
subdf.apply(lambda col: col.max() - col.min()) # axis=0 is the default

tvl          1.473266e+10
change_1d    8.810051e+01
change_7d    1.457891e+02
mcap         1.686376e+09
dtype: float64

In [7]:
# in practice, I prefer to write like this 
subdf.max() - subdf.min()

tvl          1.473266e+10
change_1d    8.810051e+01
change_7d    1.457891e+02
mcap         1.686376e+09
dtype: float64

In [8]:
# extract protocols with the largest TVL, 1 and 7 day change in TVL, or MCap
subdf.apply(lambda col: col.idxmax())

tvl                   Lido
change_1d    Collectif DAO
change_7d    Collectif DAO
mcap                  Lido
dtype: object

In [9]:
# once again, please write like this 
subdf.idxmax()

tvl                   Lido
change_1d    Collectif DAO
change_7d    Collectif DAO
mcap                  Lido
dtype: object

In [10]:
# because not only it's simpler, it's also faster
%timeit subdf.idxmax()
%timeit subdf.apply(lambda col: col.idxmax())

365 µs ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
701 µs ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


So far, I've only given simple examples where `apply()` is not needed and it's
better to leave it out. I intend to get you familar with the use pattern: apply 
a function to all columns (or all rows). Let's now see a couple of more involved 
examples.

In [11]:
# apply the following logic to TVL and Mcap: 
# if value > 500M, then "500M+", else "500M-"
tvl_mcap_gps = subdf[['tvl', 'mcap']].apply(
    # np.where is vectorized, and it applies comparison to entire col
    lambda col: np.where(col >= 500*1e6, "500M+", "500M-")  
)
tvl_mcap_gps

,tvl,mcap
name,,
Lido,500M+,500M+
Coinbase Wrapped Staked ETH,500M+,500M-
Rocket Pool,500M+,500M+
Frax Ether,500M-,500M-
StakeWise,500M-,500M-
...,...,...
Interlay Staking,500M-,500M-
NUDES,500M-,500M-
GhostMarket,500M-,500M-


In [12]:
# # uncomment this block and run it to receive an error

# subdf[['tvl', 'mcap']].apply(
#     # using if-else (instead of np.where) throws an error because if-else is not 
#     # vectorized and it operates one value (scalar) at a time,
#     # to make it work, you'd need to use apply() twice. Don't do it!   
#     lambda x: "500M+" if x >= 500*1e6 else "500M-"
# )

In [13]:
# you can also do it with vectorization but you lose the header and index
def bin_var(col): 
    return np.where(col >= 500*1e6, "500M+", "500M-")  
pd.DataFrame(bin_var(subdf[['tvl', 'mcap']]))

,0,1
0,500M+,500M+
1,500M+,500M-
2,500M+,500M+
3,500M-,500M-
4,500M-,500M-
...,...,...
101,500M-,500M-
102,500M-,500M-
103,500M-,500M-
104,500M-,500M-


In [14]:
# apply the following logic to 1d and 7d changes: 
#   delta > 50% => "big jump",
#   delta > 0 and <= 50% => "jump",
#   delta > -50% and <= 0 => 'drop"
#   delta <= -50% => 'big drop"

# translate the logic into bins and bin labels cuz we'll use pd.cut()
# by default, the resulting bins include the right edge not the left one
bins = [-np.inf, -.5, 0, .5, np.inf]
labs = ['big drop', 'drop', 'jump', 'big jump']
tvl_delta_gps = subdf[['change_1d', 'change_7d']].apply(
    lambda col: pd.cut(col, bins=bins, labels=labs)
)
tvl_delta_gps

,change_1d,change_7d
name,,
Lido,jump,big drop
Coinbase Wrapped Staked ETH,jump,big drop
Rocket Pool,jump,big drop
Frax Ether,jump,big drop
StakeWise,jump,big drop
...,...,...
Interlay Staking,NaN,NaN
NUDES,NaN,NaN
GhostMarket,NaN,NaN


In [15]:
# # try to do it naively and hope vectorization works. Oops, error!
# pd.cut(subdf[['change_1d', 'change_7d']], bins=bins, labels=labs)

### `groupby()` and `apply()`

It is very common to use `apply()` on a group of DataFrames. So you'll write a 
lot of `groupby()`s followed by `apply()`, especially when applying your custom
functions. And most of the time, the function you apply to each group is a 
reducing function (i.e., reduces data dimension), for example, an aggregation 
function such as `mean()` is a reducing function, taking the first element of 
an array is also a reducing function although it aggregates nothing. 

In [16]:
# group by 1d change levels ('big drop', 'drop', 'jump', 'big jump') and 
# find the protocol with the max TVL, 1d or 7d TVL change, or Mcap for each level
subdf.groupby(tvl_delta_gps['change_1d'])\
    .apply(lambda da: da.idxmax()) # think of this syntax as iterating over 
    # each group and appling the lambda function column-wise to the chunk of 
    # data (a data frame) in that group.

,tvl,change_1d,change_7d,mcap
change_1d,,,,
big drop,Stader,Tenderize,Filet Finance,Stride
drop,Marinade Finance,Marinade Finance,sICX,Stafi
jump,Lido,Ankr,SFT Protocol,Lido
big jump,Benqi Staked Avax,Collectif DAO,Collectif DAO,Veno Finance


Among all protocols with a big (-50% or more) TVL drop in one day, Stride has the largest TVL and Mcap, 
IFPool has the largest single day drop in TVL, and ClayStack has the largest 7d drop in TVL.

Among all protocols with a big (50% or more) TVL jump in one day, Coinbase staked ETH has the largest TVL,
Ankr has the largest Mcap, Neopin has the largest 1d increase in TVL and xALGO has the largest 7d increase.

In [17]:
# group by mcap levels (500M-, 500M+) and 
# calculate the ratio between the avg 7d tvl change and the avg 1d tvl change
subdf.groupby(tvl_mcap_gps['mcap'])\
    .apply(lambda da: da['change_7d'].mean() / da['change_1d'].mean())

mcap
500M+   -6.373806
500M-   -2.195599
dtype: float64

In [18]:
# you can also do it this way
(subdf.groupby(tvl_mcap_gps['mcap'])['change_7d'].mean() / 
 subdf.groupby(tvl_mcap_gps['mcap'])['change_1d'].mean())

mcap
500M+   -6.373806
500M-   -2.195599
dtype: float64

In [19]:
# which is faster?
%timeit subdf.groupby(tvl_mcap_gps['mcap']).apply(lambda da: da['change_7d'].mean() / da['change_1d'].mean())
%timeit subdf.groupby(tvl_mcap_gps['mcap'])['change_7d'].mean() / subdf.groupby(tvl_mcap_gps['mcap'])['change_1d'].mean()

1.97 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.81 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Let's see some non-trivial examples where we'll apply string operations column wise to a grouped data frame. 
Let's first download liquidity pools' yield data from DefiLlama.

In [20]:
da = obj.get_pools_yields()

# only pick these cols
cols = ['chain', 'project', 'symbol', 'stablecoin', 'tvlUsd', 'apyPct7D', 'outlier']
da = da[cols]
da.head()

,chain,project,symbol,stablecoin,tvlUsd,apyPct7D,outlier
0,Ethereum,lido,STETH,False,14577608773,0.20000,False
1,Arbitrum,camelot-v2,BRUH-USDC,False,3055925878,0.00000,False
2,Arbitrum,camelot-v2,RDO-USDC,False,2995936715,0.00000,True
3,Tron,justlend,BTC,False,2970582736,0.00045,False
4,Ethereum,coinbase-wrapped-staked-eth,CBETH,False,2170167164,0.20627,False


We can then use `groupby()` + `apply()` to find which chains have the most ETH related yield opportunities.

In [21]:
# find top 10 chains with the most ETH related yield opportunities
da.groupby('chain', sort=False)['symbol']\
    .apply(lambda ser: ser.str.contains('ETH').sum()
).nlargest(10)

chain
Ethereum         2584
Arbitrum          991
Polygon           394
Optimism          231
BSC               142
Avalanche          43
Fantom             33
Solana             18
Gnosis             15
Polygon zkEVM      14
Name: symbol, dtype: int64

To see what's going on, let's break things down. The `groupby()` function groups 
the records and results in a data frame for each "chain". The `[symbol]` operation 
extracts the "symbol" column and returns it as a series. The lambda function first 
checks if each value of the series contains the word 'ETH' and then sum them up 
to get a count. Finally, the `apply()` function applies the lambda function to 
each column, and we only have 1 column here, the 'symbol' column. Let's apply 
the string operation manually to the first data frame in the group.

In [22]:
# groups are sorted ascendingly by default, we turn it off here by setting
# `sort=False` inside groupby()
chain, ser = next(iter(da.groupby('chain', sort=False)['symbol'])) 
print(chain)
print(ser.head())


Ethereum
0     STETH
4     CBETH
6     STETH
7      RETH
8    WSTETH
Name: symbol, dtype: object


In [23]:
ser.str.contains('ETH').sum()

2584

We see ETH mainnet has more than 2500 yield opportunities in ETH-related assets. The above approach is good but there's a faster way. We can first check if the 'symbol' column contains the word 'ETH' in one-go because `ser.str.contains()` is vectorized and then use `groupby()`. 

In [24]:
# this approach is faster cuz it leverages vectorization
is_eth_related = da['symbol'].str.contains('ETH') # a bool series of True and False
is_eth_related.groupby(da['chain'], sort=False)\
    .sum()\
    .nlargest(10)\
    .astype(np.uintc) # cast to unsigned int to make it compact

chain
Ethereum         2584
Arbitrum          991
Polygon           394
Optimism          231
BSC               142
Avalanche          43
Fantom             33
Solana             18
Gnosis             15
Polygon zkEVM      14
Name: symbol, dtype: uint32

### Summary

This notebook aims to demonstrate the most common use pattern of `apply()`: 

*Apply a reducing function either column-wise (i.e., for every column and cross 
all rows, `axis=0`) or row-wise (i.e., for every row and cross all columns, `axis=1`) 
on a DataFrame or a group of DataFrames.*

I believe mastery of this pattern allows you to handle 80% of data analysis 
projects you do. I did not give examples of using `apply()` row-wise (`axis=1`), 
can you come up with such examples? 


### Good Read

- You can get defi data easily using [defillama2](https://github.com/coindataschool/defillama2). 
- [`apply()` basics from the official doc](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#row-or-column-wise-function-application).
- [realpython tutorial: pandas groupby](https://realpython.com/pandas-groupby/).

### Referral

- Digital Ocean is a cloud computing platform where you can rent remote servers for cheap. 
  I have my remote data science server there. You can do the same and [get $200 credit](https://m.do.co/c/0a435cb96813). 